# Search PubMed and save results

This example demonstrates the typical workflow to search PubMed and store
the results in a database

Authors: Isabel Restrepo <br>
BCBI - Brown University <br>
Version: Julia 0.6

In [1]:
using BioMedQuery.Processes
using BioMedQuery.PubMed

In [2]:
email= "" #Only needed if you want to contact NCBI with inqueries
search_term="(obesity[MeSH Major Topic]) AND (\"2010\"[Date - Publication] : \"2012\"[Date - Publication])"
max_articles = 20
results_dir = "./results";

## Save to MySQL

In [3]:
mysql_config = Dict(:host=>"localhost",
                    :dbname=>"pubmed_obesity_2010_2012",
                    :username=>"root",
                    :pswd=>"",
                    :overwrite=>true)
verbose = false
db = pubmed_search_and_save(email, search_term, max_articles,
    save_efetch_mysql, mysql_config, verbose)

Getting 20 articles, starting at index 0
------ESearch--------
------EFetch--------
------Save to database--------
Initializing MySQL Database
Set to overwrite MySQL database pubmed_obesity_2010_2012
Database pubmed_obesity_2010_2012 created and initialized
Saving 20 articles to database
Finished searching, total number of articles: 20


MySQL Handle
------------
Host: localhost
User: root
DB:   pubmed_obesity_2010_2012


### Access all PMIDS

In [4]:
display(all_pmids(db))

,pmid
1,23814963
2,23823925
3,23834543
4,23834547
5,23856452
6,23857708
7,23903497
8,23923596
9,24002403
10,24002404


### Explore tables

* You may use the MySQL command directly or you can use BioMedQuery.DBUtils for a common interface to MySQL/SQLite

In [5]:
tables = ["author", "author2article", "mesh_descriptor",
"mesh_qualifier", "mesh_heading"]

for t in tables
    query_str = "SELECT * FROM "*t*" LIMIT 5;"
    q = BioMedQuery.DBUtils.db_query(db, query_str)
    println(q)
end

Any[5×3 DataFrames.DataFrame
│ Row │ id │ forename            │ lastname           │
├─────┼────┼─────────────────────┼────────────────────┤
│ 1   │ 31 │ "A"                 │ "Carbonell-Baeza"  │
│ 2   │ 88 │ "Adelina D"         │ "Tsakova"          │
│ 3   │ 26 │ "Alexandre Paulino" │ "de Faria"         │
│ 4   │ 48 │ "Alvaro"            │ "Mathiew-Quirós"   │
│ 5   │ 46 │ "Ana Maria"         │ "Salinas-Martínez" │]
Any[5×2 DataFrames.DataFrame
│ Row │ aid │ pmid     │
├─────┼─────┼──────────┤
│ 1   │ 91  │ 23814963 │
│ 2   │ 92  │ 23814963 │
│ 3   │ 88  │ 23823925 │
│ 4   │ 89  │ 23823925 │
│ 5   │ 90  │ 23823925 │]
Any[5×2 DataFrames.DataFrame
│ Row │ id    │ name             │
├─────┼───────┼──────────────────┤
│ 1   │ 50153 │ "Abdominal Fat"  │
│ 2   │ 54392 │ "Adipokines"     │
│ 3   │ 52242 │ "Adiponectin"    │
│ 4   │ 273   │ "Adipose Tissue" │
│ 5   │ 50154 │ "Adiposity"      │]
Any[5×2 DataFrames.DataFrame
│ Row │ id  │ name                    │
├─────┼─────┼────────────────